## throw vs throws — explanation, syntax, examples
### throw

What: A statement to explicitly throw an exception object from a method or block.

Used to: Signal an error condition immediately (built-in or custom exception).

Affects control flow: Immediately transfers control to nearest matching catch or up the call stack.

Syntax: throw new SomeException("message");

Example:

In [ ]:
void checkAge(int age) {
    if (age < 18) {
        throw new IllegalArgumentException("Age must be >= 18");
    }
}


Notes: throw always throws a single instance (object) — you must new it or rethrow an existing throwable.

throws

What: A method declaration clause that tells callers the method might throw those checked exceptions.

Used to: Propagate responsibility to the caller (caller must handle or further declare).

Syntax: void read() throws IOException, SQLException { ... }

Example:

In [ ]:
void readFile() throws IOException {
    FileReader fr = new FileReader("file.txt"); // may throw FileNotFoundException
    // ...
}


Notes: throws is only about checked exceptions (compiler-checked). You may declare unchecked exceptions too, but it is optional.

| Aspect                   |                                   `throw` | `throws`                                                           |
| ------------------------ | ----------------------------------------: | ------------------------------------------------------------------ |
| Kind of construct        |             Statement inside method/block | Method signature modifier                                          |
| Purpose                  | Actually create/throw an exception object | Declare that method may pass exception to caller                   |
| When checked by compiler |                     Runtime when executed | Compile-time: forces callers to handle declared checked exceptions |
| Where used               |                       Inside method/block | In method declaration after parameter list                         |
| Example                  |             `throw new IOException("x");` | `void m() throws IOException { ... }`                              |


### try-catch vs throws (handling vs declaring)

#### try-catch

What: Mechanism to handle exceptions immediately where they may occur.

When to use: When you can meaningfully recover, log properly, or provide a fallback.

Effect: Exception is consumed (unless rethrown) and control continues after catch/finally.

#### throws

What: Lets the method delegate exception handling to its caller.

When to use: When the method cannot or should not decide how to handle the problem; higher-level context should decide.

Effect: Compiler enforces handling for checked exceptions at some higher level.

Guideline: catch where you can handle it sensibly. declare (throws) when caller should handle it.

Example showing both:

In [ ]:
void lowLevel() throws IOException {
    // I/O code that may fail
    throw new IOException("disk failed");
}

void midLevel() {
    try {
        lowLevel();
    } catch (IOException e) {
        // recover or convert to a runtime exception
        System.err.println("Recovered in midLevel: " + e.getMessage());
    }
}


## ⭐ User-Defined (Custom) Exception – Explained

Java already has many built-in exceptions like
ArithmeticException, NullPointerException, IOException, etc.

But sometimes we need our own exception to represent a custom error situation.

Example:

Age below 18 → Not eligible

Invalid marks → Should not be >100

Insufficient balance in ATM

Invalid username/password

In such cases, we create our own exception class.

Custom exception must extend Exception (checked) or RuntimeException (unchecked).

### ⭐ Easy & Perfect Example: Age Validation

We will create:

A custom exception → AgeException

A method that throws the custom exception

A try–catch to handle it

In [ ]:
// Step 1: Create a custom exception class
class AgeException extends Exception {

    // Constructor to pass custom message
    public AgeException(String message) {
        super(message);
    }
}

// Step 2: A method that checks age and throws custom exception
class VotingSystem {

    public static void checkAge(int age) throws AgeException {
        if (age < 18) {
            throw new AgeException("Age is below 18. Not eligible to vote.");
        } else {
            System.out.println("Eligible to vote!");
        }
    }
}

// Step 3: Main class to test our custom exception
public class Main {
    public static void main(String[] args) {
        try {
            VotingSystem.checkAge(15);   // Passing an invalid age
        }
        catch (AgeException e) {
            System.out.println("Custom Exception Caught: " + e.getMessage());
        }
    }
}


#### ⭐ Explanation (Simple and Clear)
✔ Step 1: Creating the Custom Exception Class


In [ ]:
class AgeException extends Exception

We create a class AgeException.

It extends Exception, meaning it is a checked exception.

It contains a constructor to pass a custom message.

✔ Step 2: Throwing the Exception

In [ ]:
if (age < 18) {
    throw new AgeException("Age is below 18.");
}



If age < 18, we manually throw our custom exception using throw.

The method declares:

In [ ]:
throws AgeException


so that the compiler knows this method may throw an exception.

✔ Step 3: Handling the Exception

In [ ]:
catch (AgeException e)


We catch the custom exception.

e.getMessage() prints the message we passed.

#### ⭐ Program Output

In [ ]:
Custom Exception Caught: Age is below 18. Not eligible to vote.


## 📌 Exception Propagation in Java

Exception propagation means how an exception moves (propagates) from one method to another in the call stack until it is handled by a suitable catch block.

In Java, if a method does not handle an exception, the exception is thrown to its caller.
If the caller also doesn’t handle it, the exception keeps moving upward through the call stack.

If no method handles it → the program terminates with an error.

### 🔥 How Propagation Works

If method A() calls B(), and B() calls C(), and C() throws an exception:

In [ ]:
A() → calls B()
B() → calls C()
C() → exception is thrown


Propagation flow:

In [ ]:
Exception in C()
       ↓
goes to B()
       ↓
goes to A()
       ↓
program ends if not caught


### ⭐ Example of Exception Propagation

In [ ]:
public class PropagationDemo {

    static void methodC() {
        // Exception originates here
        System.out.println("Inside methodC");
        int a = 10 / 0;  // ArithmeticException
    }

    static void methodB() {
        System.out.println("Inside methodB");
        methodC(); // call methodC
    }

    static void methodA() {
        System.out.println("Inside methodA");
        methodB(); // call methodB
    }

    public static void main(String[] args) {
        System.out.println("Main started");

        try {
            methodA();   // start the chain
        } 
        catch (Exception e) {
            System.out.println("Exception caught in main: " + e);
        }

        System.out.println("Program continues...");
    }
}


#### ⭐ Explanation (Easy)

method3() makes an error → division by zero

It doesn’t handle the exception

So it gets passed to method2()

method2() also does not handle → passed to method1()

method1() does not handle → passed to main()

#### ✔ Program Output

In [ ]:
Exception handled in main(): java.lang.ArithmeticException: / by zero
Program continues...
